In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
import os 

2025-07-03 11:28:13.557015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751542094.008489      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751542094.118908      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# --- Parameters ---#
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
IMG_SHAPE = IMAGE_SIZE + (3,)
NUM_CLASSES = 2
EPOCHS = 50

In [3]:
# --- Directories ---#
train_dir = r"/kaggle/input/facecom/Comys_Hackathon5/Task_A/train"
val_dir = r"/kaggle/input/facecom/Comys_Hackathon5/Task_A/val"

In [4]:
# --- Data Generators ---#
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [6]:

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42,
    classes=['female', 'male']  
)

Found 1926 images belonging to 2 classes.


In [7]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    classes=['female', 'male']
)

Found 422 images belonging to 2 classes.


In [8]:
# --- Model Architecture ---#
inputs = tf.keras.Input(shape=IMG_SHAPE)
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SHAPE,
    pooling='avg'
)(inputs)
x = layers.Dropout(0.5)(base_model)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

I0000 00:00:1751542116.550525      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751542116.551209      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
# --- Focal Loss ---#
def focal_loss(gamma=2.0, alpha=0.75):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * y_true * tf.pow(1 - y_pred, gamma)
        return tf.reduce_sum(weight * cross_entropy, axis=1)
    return loss

In [10]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss=focal_loss(gamma=2.0, alpha=0.8),
    metrics=['accuracy', tf.keras.metrics.Recall(name='recall')]
)

In [11]:
# --- Initial Training ---#
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_recall', patience=5, restore_best_weights=True, mode='max'),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
    ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1751542180.085702     113 service.cc:148] XLA service 0x7e0ef4004d80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751542180.087161     113 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751542180.087189     113 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751542185.529254     113 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1751542196.887509     113 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751542197.031849     113 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751542197.485280     113 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. Th

41/61 ━━━━━━━━━━━━━━━━━━━━ 12s 626ms/step - accuracy: 0.6252 - loss: 0.1590 - recall: 0.6252

E0000 00:00:1751542256.058567     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751542256.194392     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


61/61 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.6679 - loss: 0.1436 - recall: 0.6679 - val_accuracy: 0.1872 - val_loss: 0.3350 - val_recall: 0.1872 - learning_rate: 1.0000e-04
Epoch 2/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 31s 511ms/step - accuracy: 0.8817 - loss: 0.0622 - recall: 0.8817 - val_accuracy: 0.1872 - val_loss: 0.4487 - val_recall: 0.1872 - learning_rate: 1.0000e-04
Epoch 3/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 30s 490ms/step - accuracy: 0.9079 - loss: 0.0486 - recall: 0.9079 - val_accuracy: 0.1872 - val_loss: 0.3353 - val_recall: 0.1872 - learning_rate: 1.0000e-04
Epoch 4/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 31s 501ms/step - accuracy: 0.9121 - loss: 0.0464 - recall: 0.9121 - val_accuracy: 0.2038 - val_loss: 0.2272 - val_recall: 0.2038 - learning_rate: 5.0000e-05
Epoch 5/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 30s 497ms/step - accuracy: 0.9178 - loss: 0.0453 - recall: 0.9178 - val_accuracy: 0.2725 - val_loss: 0.2674 - val_recall: 0.2725 - learning_rate: 5.0000e-05
Epoch 6/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 

In [12]:
def print_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    # Calculate max length for formatting
    max_len = max(len(name) for name in class_names) + 5
    
    # Create header
    header = " " * max_len + "| " + " | ".join([f"Predicted {name}" for name in class_names])
    separator = "-" * len(header)
    
    # Create rows
    rows = []
    for i, true_name in enumerate(class_names):
        row = f"True {true_name}".ljust(max_len) + "| "
        row += " | ".join([f"{cm[i,j]:<{len('Predicted ' + class_names[j])}}" for j in range(len(class_names))])
        rows.append(row)
    
    # Print matrix
    print("\nConfusion Matrix:")
    print(header)
    print(separator)
    for row in rows:
        print(row)

In [13]:
# --- Final Evaluation ---#
y_true = val_generator.classes
y_pred = model.predict(val_generator).argmax(axis=1)
class_names = ['Female', 'Male']

14/14 ━━━━━━━━━━━━━━━━━━━━ 16s 721ms/step


In [14]:
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

print_confusion_matrix(y_true, y_pred, ['Female', 'Male'])

              precision    recall  f1-score   support

      Female     0.8657    0.7342    0.7945        79
        Male     0.9408    0.9738    0.9570       343

    accuracy                         0.9289       422
   macro avg     0.9033    0.8540    0.8758       422
weighted avg     0.9268    0.9289    0.9266       422


Confusion Matrix:
           | Predicted Female | Predicted Male
----------------------------------------------
True Female| 58               | 21            
True Male  | 9                | 334           


In [15]:
hard_images, hard_labels = [], []

for i in range(len(val_generator)):
    images, labels = val_generator[i]
    preds = model.predict(images, verbose=0)
    pred_classes = np.argmax(preds, axis=1)
    true_classes = np.argmax(labels, axis=1)
    
    for j in range(len(images)):
        if pred_classes[j] != true_classes[j]:
            hard_images.append(images[j])
            hard_labels.append(labels[j])
    
    if len(hard_images) > 100: 
        break

hard_images = np.array(hard_images)
hard_labels = np.array(hard_labels)

print(f"Collected {len(hard_images)} hard samples for fine-tuning")

Collected 30 hard samples for fine-tuning


In [16]:
hard_images = np.array(hard_images)
hard_labels = np.array(hard_labels)

print(f"Collected {len(hard_images)} hard samples for fine-tuning")

Collected 30 hard samples for fine-tuning


In [17]:
if len(hard_images) > 0:
    # Create dataset for hard samples#
    hard_dataset = tf.data.Dataset.from_tensor_slices((hard_images, hard_labels))
    hard_dataset = hard_dataset.shuffle(len(hard_images)).batch(8)
    
    # Lower learning rate for fine-tuning#
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),
        loss=focal_loss(gamma=2.0, alpha=0.75),
        metrics=['accuracy']
    )
    
    print("\nFine-tuning on hard samples...")
    model.fit(
        hard_dataset,
        epochs=5,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
    )


Fine-tuning on hard samples...
Epoch 1/5


E0000 00:00:1751542982.420046     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751542982.557070     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


4/4 ━━━━━━━━━━━━━━━━━━━━ 115s 9s/step - accuracy: 0.2900 - loss: 1.1678
Epoch 2/5
3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.2222 - loss: 1.1001

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.2533 - loss: 1.0997
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2225 - loss: 1.1739
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.2442 - loss: 1.1282
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3033 - loss: 0.9229


In [18]:
y_true = val_generator.classes
y_pred = model.predict(val_generator).argmax(axis=1)
class_names = ['Female', 'Male']

14/14 ━━━━━━━━━━━━━━━━━━━━ 16s 712ms/step


In [19]:
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

print_confusion_matrix(y_true, y_pred, ['Female', 'Male'])

              precision    recall  f1-score   support

      Female     0.8714    0.7722    0.8188        79
        Male     0.9489    0.9738    0.9612       343

    accuracy                         0.9360       422
   macro avg     0.9101    0.8730    0.8900       422
weighted avg     0.9344    0.9360    0.9345       422


Confusion Matrix:
           | Predicted Female | Predicted Male
----------------------------------------------
True Female| 61               | 18            
True Male  | 9                | 334           


In [20]:
y_probs = model.predict(val_generator)
female_probs = y_probs[:, 0]  
female_true = (y_true == 0).astype(int)

14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step


In [21]:
precision, recall, thresholds = precision_recall_curve(female_true, female_probs)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_idx = np.argmax(f1_scores)
female_thresh = thresholds[best_idx]
print(f"Optimal threshold for female: {female_thresh:.4f}")

Optimal threshold for female: 0.4722


In [22]:
# Apply threshold #
y_pred_adjusted = np.where(female_probs >= female_thresh, 0, 1)
print("\n=== Threshold-Adjusted Evaluation ===")
print(classification_report(y_true, y_pred_adjusted, target_names=class_names, digits=4))
print_confusion_matrix(y_true, y_pred_adjusted, ['Female', 'Male'])


=== Threshold-Adjusted Evaluation ===
              precision    recall  f1-score   support

      Female     0.8272    0.8481    0.8375        79
        Male     0.9648    0.9592    0.9620       343

    accuracy                         0.9384       422
   macro avg     0.8960    0.9036    0.8997       422
weighted avg     0.9390    0.9384    0.9387       422


Confusion Matrix:
           | Predicted Female | Predicted Male
----------------------------------------------
True Female| 67               | 12            
True Male  | 14               | 329           


In [24]:
# Create the models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Save weights with the correct filename ending
model.save_weights('models/gender_classifier.weights.h5')

# Save the full model (architecture + weights + optimizer state)
model.save('models/gender_classifier_full.h5')

In [26]:
# After finding the best threshold
female_thresh = thresholds[best_idx]
with open('models/female_threshold.txt', 'w') as f:
    f.write(str(female_thresh))